In [1]:
import rpy2.robjects.packages as rpackages
from rpy2.rlike.container import TaggedList
from rpy2.robjects.vectors import StrVector, ListVector

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import weighted_imputation as wi

In [3]:
BASE_PATH = os.path.abspath('../weighted_imputation/networks')

Getting ready to install packages from the first mirror known to R

In [4]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f9f440d71e0> [RTYPES.NILSXP]

List of packages to install if not already installed

In [5]:
packages = ('bnlearn', 'gRain')
packages_to_install = [
    x
    for x in packages
    if not rpackages.isinstalled(x)
]
if len(packages_to_install) > 0:
    utils.install_packages(StrVector(packages_to_install))

In [6]:
base = rpackages.importr('base')
bnlearn = rpackages.importr('bnlearn')
gRain = rpackages.importr('gRain')

In [7]:
network = 'asia.bif'
network = os.path.join(BASE_PATH, network)
network

'/home/alessio/madlab_weighted_imputation/weighted_imputation/networks/asia.bif'

In [8]:
network = bnlearn.read_bif(network)
network

R object with classes: ('bn.fit', 'bn.fit.dnet') mapped to:
[Lis..., Lis..., Lis..., Lis..., Lis..., Lis..., Lis..., Lis...]
  asia: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee1584910> [RTYPES.VECSXP]
  tub: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee151f0a0> [RTYPES.VECSXP]
  smoke: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee1584910> [RTYPES.VECSXP]
  lung: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee151fb40> [RTYPES.VECSXP]
  bronc: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee151f0a0> [RTYPES.VECSXP]
  either: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee151fb40> [RTYPES.VECSXP]
  xray: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee151f0a0> [RTYPES.VECSXP]
  dysp: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7f9ee151fb40> [RTYPES.VECSXP]

In [9]:
evidence = {'lung': 'yes'}
evidence = ListVector(evidence)
evidence

lung,[RTYPES.STRSXP]


In [10]:
network = bnlearn.mutilated(network, evidence)

In [11]:
network = bnlearn.as_grain(network)

In [12]:
nodes = ['asia', 'xray']
qtype = 'marginal'
query = gRain.querygrain(
    network,
    nodes = StrVector(nodes),
    type = qtype,
    result = 'data.frame'
)
query

asia,[RTYPES.VECSXP]
xray,[RTYPES.VECSXP]


In [13]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

out = []
for i, node in enumerate(nodes):
    with localconverter(ro.default_converter + pandas2ri.converter):
        q = ro.conversion.rpy2py(query[i])
        q = q.set_index(node).to_xarray()
        q = q.to_array().squeeze(['variable'], drop=True)
        out.append(q)
out

[<xarray.DataArray (asia: 2)>
 array([0.01, 0.99])
 Coordinates:
   * asia     (asia) object 'yes' 'no',
 <xarray.DataArray (xray: 2)>
 array([0.98, 0.02])
 Coordinates:
   * xray     (xray) object 'yes' 'no']